In [ ]:
import boto3
from datetime import datetime
import pandas as pd

### ==============================================================================================

In [ ]:
env = 'dev'
CACHED_S3_BUCKET = 'saiva-dev-data-bucket'
client = 'trio'
task_name = 'saiva-3-day-hosp-v1'
facilities = [1, 7, 21, 42, 52, 55, 186, 194, 265, 273, 274, 275, 276, 277, 278, 279]

In [ ]:
env = 'staging'
CACHED_S3_BUCKET = 'saiva-staging-data-bucket'
client = 'avante'
task_name = 'saiva-3-day-hosp-v1'
# facilities = [1, 3, 4, 5, 6, 7, 8, 9, 10, 13, 21]
facilities = [7]

In [ ]:
env = 'prod'
CACHED_S3_BUCKET = 'saiva-prod-data-bucket'
client = 'dycora'
task_name = 'dycora-3-day-hosp'
# facilities = [121, 302]
facilities = [302]

In [ ]:
AutoScalingGroupName = {
    "dev": "dev-Internal-ASG",
    "staging": "staging-Internal-ASG",
    "prod": "prod-Internal-ASG"
}

ClusterName = {
    "dev": "dev-Internal",
    "staging": "staging-Internal",
    "prod": "prod-Internal"  
}

networkConfiguration = {
    "dev": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-008159c0210f49f96'], 
            'subnets': ['subnet-0eb41526eff222cab', 'subnet-04b6e7f553bd31909'], 
            'assignPublicIp': 'DISABLED'}},
    "staging": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-0f8d382201750604e'], 
            'subnets': ['subnet-0b9343ae7c5712608', 'subnet-01d3901613027f4d9'], 
            'assignPublicIp': 'DISABLED'}},
    "prod": {
            "awsvpcConfiguration": {
                "securityGroups": ["sg-041a9f4c8e2f4fd56"],
                "subnets": ["subnet-0c87ee56c3e4e27d1", "subnet-065f42df2509bd9f5"],
                "assignPublicIp": "DISABLED",
            }
        }
}

In [ ]:
pred_date_list = pd.date_range(start='9/15/2020', end='9/21/2020')
pred_date_list

In [ ]:
desired_count = len(pred_date_list)
desired_count = desired_count * len(facilities)
desired_count

In [ ]:
asg = boto3.client('autoscaling', region_name='us-east-1')
# Scale Up the ASG
response = asg.update_auto_scaling_group(
        AutoScalingGroupName=AutoScalingGroupName[env],
        DesiredCapacity=desired_count,
        MaxSize=desired_count,
        DefaultCooldown=100
    )
response

In [ ]:
ecs = boto3.client('ecs', region_name='us-east-1')

In [ ]:
def start_backfill(facility_id, prediction_date):
    response = ecs.run_task(
        taskDefinition=f"{env}-{task_name}",
        cluster=f'{env}-Internal',
        overrides={
            "containerOverrides": [
                {
                    "name": f"{task_name}",
                    "command": [
                        "/bin/bash",
                        "-c",
                        f"python /src/run_model.py --client {client} --facilityids '[{facility_id}]' --prediction-date '{prediction_date}' --s3-bucket {CACHED_S3_BUCKET} --replace_existing_predictions True --cache_strategy ignore_cache",
                    ],
                }
            ]
        },
        launchType="EC2",
        networkConfiguration=networkConfiguration[env],
        placementStrategy=[
            {
                "field": "instanceId",
                "type": "spread"
            }
        ]
    )
    return response

In [ ]:
# Trigger run_model task for different prediction dates
result_list = []
for pred_date in pred_date_list:
    pred_date_str = pred_date.strftime('%Y-%m-%d')
    for facility in facilities:
        results = start_backfill(facility_id=facility, prediction_date=pred_date_str)
        result_list.append(results)
print(result_list)
job_start_time = datetime.now()
dt_string = job_start_time.strftime("%Y-%m-%d %H:%M:%S")
print(f'started Jobs on {dt_string}')

In [ ]:
# Print results of jobs, so we know when to stop ASG
# example: print(result_list[0]['tasks'][0]['containers'][0]['taskArn'])
count = 0
count_not_finished = 0
num_failed = 0
for tasks in result_list:
    task_arn = tasks['tasks'][0]['containers'][0]['taskArn']
    print(f'{count}. task_arn={task_arn}')  
    task_list = []
    task_list.append(task_arn)
    task_desc = ecs.describe_tasks(cluster=ClusterName[env], tasks=task_list)
    exit_code = task_desc['tasks'][0]['containers'][0].get('exitCode')
    if (exit_code is None):
        print('\t task is still running!!')
        count_not_finished += 1
    elif exit_code == 0:
        print(f'\t task successfully finished')
    else:
        print(f'\t task failed.  exit code was {exit_code}!!')
        num_failed += 1
    count += 1

print('-------------------')
current_time = datetime.now()
dt_string = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f'Current time: {dt_string}')
time_diff = current_time - job_start_time
print(f'Time taken so far: {time_diff.total_seconds()/60:.0f} minutes')
      
if num_failed != 0:
    print(f'Failure count: {num_failed}')
else:
    print('No Failures - woo hoo!')
if count_not_finished != 0:
    print(f'Summary: Hold your horses! There are still {count_not_finished} jobs to be done')
else:
    print('Summary: ALL DONE!!')
print('-------------------')

In [ ]:
# Only Execute after all jobs are successfully finished!
asg = boto3.client('autoscaling', region_name='us-east-1')
# Scale DOWN the ASG.
response = asg.update_auto_scaling_group(
        AutoScalingGroupName=AutoScalingGroupName[env],
        DesiredCapacity=0,
        MaxSize=0,
        DefaultCooldown=100
    )
response

## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
import boto3
from datetime import datetime


In [ ]:
client = 'avante'
env = 'dev'
facilities = [1,3,4,5,6,7,8,9,10,13,21]
CACHED_S3_BUCKET = 'saiva-dev-data-bucket'
task_name = 'saiva-3-day-hosp-v1'
prediction_start_date = '2021-01-01'
prediction_end_date = '2021-01-31'

saiva = boto3.session.Session()
boto3.setup_default_session()

In [ ]:
vpn_instance = {
    "dev": "i-07197a52885c34d68",
    "staging": "i-07d9c5587e7637960",
    "prod": "i-0e5f01117bb50fdb1"
}

AutoScalingGroupName = {
    "dev": "dev-Internal-ASG",
    "staging": "staging-Internal-ASG",
    "prod": "prod-Internal-ASG"
}

ClusterName = {
    "dev": "dev-Internal",
    "staging": "staging-Internal",
    "prod": "prod-Internal"  
}

networkConfiguration = {
    "dev": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-008159c0210f49f96'], 
            'subnets': ['subnet-0eb41526eff222cab', 'subnet-04b6e7f553bd31909'], 
            'assignPublicIp': 'DISABLED'}},
    "staging": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-0f8d382201750604e'], 
            'subnets': ['subnet-0b9343ae7c5712608', 'subnet-01d3901613027f4d9'], 
            'assignPublicIp': 'DISABLED'}},
    "prod": {
            "awsvpcConfiguration": {
                "securityGroups": ["sg-041a9f4c8e2f4fd56"],
                "subnets": ["subnet-0c87ee56c3e4e27d1", "subnet-065f42df2509bd9f5"],
                "assignPublicIp": "DISABLED",
            }
        }
}

In [ ]:
# Turn on the Avante VPN Server 

ec2 = boto3.client('ec2', region_name='us-east-1')
ec2.start_instances(
    InstanceIds=[
        vpn_instance[env], # Instance ID of the VPN Server
    ]
)

In [ ]:
asg = boto3.client('autoscaling', region_name='us-east-1')
desired_count = len(facilities)
# Scale Up the ASG
response = asg.update_auto_scaling_group(
        AutoScalingGroupName=AutoScalingGroupName[env],
        DesiredCapacity=desired_count,
        MaxSize=desired_count,
        DefaultCooldown=100
    )
response

In [ ]:
# List all running services or Tasks

ecs = boto3.client('ecs', region_name='us-east-1')
response = ecs.list_tasks(cluster="dev-Internal",maxResults=100,launchType='EC2')
response['taskArns']

In [ ]:
def start_backfill(facility_id, prediction_start_date,prediction_end_date):
    response = ecs.run_task(
        taskDefinition=f"{env}-{task_name}",
        cluster=f'{env}-Internal',
        overrides={
            "containerOverrides": [
                {
                    "name": f"{task_name}",
                    "command": [
                        "/bin/bash",
                        "-c",
                        f"python /src/run_backfill.py --client {client} --facilityids '[{facility_id}]' --prediction_start_date '{prediction_start_date}' --prediction_end_date '{prediction_end_date}' --s3-bucket {CACHED_S3_BUCKET} --replace_existing_predictions True ",
                    ],
                }
            ]
        },
        launchType="EC2",
        networkConfiguration=networkConfiguration[env],
        placementStrategy=[
            {
                "field": "instanceId",
                "type": "spread"
            }
        ]
    )
    return response

In [ ]:
print(datetime.now())
results = [start_backfill(
    facility_id=facility, 
    prediction_start_date=prediction_start_date, 
    prediction_end_date=prediction_end_date) for facility in facilities]
results